In [1]:
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None 

def connect_prd():
    try:
        connection = psycopg2.connect(
            host="35.187.250.81",
            database="prd_order",
            user="akhiyar_waladi",
            password="nd4n6fk9")


        print("You are connected")

    except (Exception, psycopg2.Error) as error :
        print("Error while connecting to PostgreSQL", error)
        
    return connection

In [11]:
## intermezzo

to_date = datetime.now().date()
now_date = to_date - timedelta(days=30)


now_str = now_date.strftime('%Y-%m-%d')
to_str = to_date.strftime('%Y-%m-%d')
print(now_str, to_str)

2021-05-12 2021-06-11


In [12]:
li_count_order = []
# li_count_order_outlier = []
li_date = []
connection = connect_prd()
for dr in pd.date_range(now_str, to_str, freq='30min'):
    if dr.hour in [1, 2, 3, 4, 5, 6]:
        q_1 = '''
            select count(tbto_no)
            from tb_transaction_order tto 
            where tto.tbto_create_date between '{shift_str}' and '{now_str}'
            and tbto_status is not null
            and tbto_status in ('12','14','15')
        '''.format(shift_str=str(dr-timedelta(hours=1)), now_str=str(dr))
        li_date.append(dr)
        
        

        count_order = pd.read_sql(q_1, connection)
        li_count_order.append(count_order['count'][0])
#         li_count_order_outlier.append()
connection.close()

count_order_df = pd.DataFrame({'count_order':li_count_order, 'date_order':li_date})

You are connected


In [13]:
count_order_df.to_excel('order_0101_1006.xlsx', index=False)

## Order Count in Midnight (00.00 - 06.00) am

>
* every ticks represent range 30 minutes check in midnight
* full plot show 1 month data

In [14]:
import plotly.express as px
import plotly.graph_objs as go

fig = px.line(count_order_df, x='date_order', y='count_order', template='presentation', \
              )
mean_order = int(count_order_df['count_order'].mean())
fig.add_hline(y=mean_order, line_dash="dot",
              annotation_text="mean = {}".format(mean_order), 
              annotation_position="top right")

fig.update_xaxes(
    tickformat="%d%b-%H", 
    showgrid=True, gridwidth=1, gridcolor='LightPink', title=''
)
fig.update_yaxes(

    showgrid=True, gridwidth=1, gridcolor='LightPink', title='#'
)

legend_dict = \
    legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            title='',
            title_font_family="Times New Roman",
            font=dict(
                family="Courier",
                size=12,
                color="black"
            ),
            bgcolor="LightGrey",
            bordercolor="Black",
            borderwidth=1
        )

In [15]:
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', margin=\
                  {'l':70, 'r':30, 't':30, 'b':70},legend=legend_dict)

In [8]:
connection = connect_prd()
li_voucher_usage_check = []
li_voucher_count_check = []
for dr in pd.date_range(now_str, to_str, freq='D'):
    q_2 = '''
    -- check voucher amount usage and voucher code count in one day interval
    select max(sum_voucher_usage), avg(sum_voucher_usage), min(sum_voucher_usage)
    from (
    select 
        tbto_ponta_id, sum(tbto_voucher_usage) as sum_voucher_usage
    from 
        tb_transaction_order tto 
    where 
        tto.tbto_create_date between '{shift_str}' and '{now_str}'
        and tbto_ponta_id is not null
        and tbto_voucher_usage is not null
        and tto.tbto_status not in ('18','10','11')
        and tbto_status is not null
    group by tbto_ponta_id
    ) as s_v_u

    '''.format(shift_str=str(dr-timedelta(days=1)), now_str=str(dr))
    
    voucher_usage_check = pd.read_sql(q_2, connection)
    voucher_usage_check['date'] = dr
    li_voucher_usage_check.append(voucher_usage_check)
    
    q_5 = '''
    -- check voucher amount usage and voucher code count in one day interval
    select max(count_unique_voucher), avg(count_unique_voucher), min(count_unique_voucher)
    from(
        select 
            tbto_ponta_id, count(distinct(tbto_voucher_code)) as count_unique_voucher
        from 
            tb_transaction_order tto 
        where 
            tto.tbto_create_date between '{shift_str}' and '{now_str}' 
            and tbto_ponta_id is not null
            and tbto_voucher_usage is not null
            and tto.tbto_status not in ('18','10','11')
            and tbto_status is not null
        group by tbto_ponta_id
        order by count_unique_voucher desc
    ) as s_v_c

    '''.format(shift_str=str(dr-timedelta(days=1)), now_str=str(dr))
    
    voucher_count_check = pd.read_sql(q_5, connection)
    voucher_count_check['date'] = dr
    li_voucher_count_check.append(voucher_count_check)


connection.close()


voucher_usage_check = pd.concat(li_voucher_usage_check)
voucher_count_check = pd.concat(li_voucher_count_check)


You are connected


## Voucher use count in range 1 day
>
* every ticks represent range 1 day check
* full plot show 1 month data

In [9]:
import plotly.express as px
import plotly.graph_objs as go

# fig = px.line(voucher_count_check, x='date', y=voucher_count_check.iloc[:,0:2], template='seaborn', \
#               text='amount_voucher')

fig = go.Figure()
fig.add_trace(go.Scatter(
    mode = "text+lines",
    x=voucher_count_check['date'],
    y=voucher_count_check['max'],
    name='max',
    text = voucher_count_check['max'].astype(int).astype(str),
    textposition='top center',


))
fig.add_trace(go.Scatter(
    mode = "text+lines",
    x=voucher_count_check['date'],
    y=voucher_count_check['avg'],
    name='avg',
    text = voucher_count_check['avg'].astype(int).astype(str),
    textposition='top center',

))
mean_count_max = int(voucher_count_check['max'].mean())
fig.add_hline(y=mean_count_max, line_dash="dot",
              annotation_text="avg<br>{}".format(mean_count_max), 
              annotation_position="top right")


fig.update_traces(texttemplate='%{text:.2d}', 
    textposition='top center', 
    textfont_size=11,
    hovertemplate='%{x}<br>%{text}')

fig.update_xaxes(
    tickformat="%d%b",
    showgrid=True, gridwidth=1, gridcolor='LightPink', title=''
)
fig.update_yaxes(

    showgrid=True, gridwidth=1, gridcolor='LightPink', title='# use voucher'
)

legend_dict = \
    legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            title='',
            title_font_family="Times New Roman",
            font=dict(
                family="Courier",
                size=12,
                color="black"
            ),
            bgcolor="LightGrey",
            bordercolor="Black",
            borderwidth=1
        )

In [10]:
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', margin=\
                  {'l':70, 'r':30, 't':30, 'b':70},legend=legend_dict, template='presentation')


## Voucher usage (RP) sum in range 1 day
> 
* every ticks represent range 1 day check
* full plot show 1 month data

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

# fig = px.line(voucher_count_check, x='date', y=voucher_count_check.iloc[:,0:2], template='seaborn', \
#               text='amount_voucher')

fig = go.Figure()
fig.add_trace(go.Scatter(
    mode = "text+lines",
    x=voucher_usage_check['date'],
    y=voucher_usage_check['max'],
    name='max',
    text = voucher_usage_check['max'].astype(int).astype(str),
    textposition='top center',


))
fig.add_trace(go.Scatter(
    mode = "text+lines",
    x=voucher_usage_check['date'],
    y=voucher_usage_check['avg'],
    name='avg',
    text = voucher_usage_check['avg'].astype(int).astype(str),
    textposition='top center',

))

mean_usage_max = int(voucher_usage_check['max'].mean())
fig.add_hline(y=mean_usage_max, line_dash="dot",
              annotation_text="avg<br>{}".format(mean_usage_max), 
              annotation_position="top right")

fig.update_traces(texttemplate='%{text:.2s}', 
    textposition='top center', 
    textfont_size=11,
    hovertemplate='%{x}<br>%{text}')

fig.update_xaxes(
    tickformat="%d%b",
    showgrid=True, gridwidth=1, gridcolor='LightPink', title=''
)
fig.update_yaxes(

    showgrid=True, gridwidth=1, gridcolor='LightPink', title='# voucher usage'
)

legend_dict = \
    legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            title='',
            title_font_family="Times New Roman",
            font=dict(
                family="Courier",
                size=12,
                color="black"
            ),
            bgcolor="LightGrey",
            bordercolor="Black",
            borderwidth=1
        )

In [ ]:
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', margin=\
                  {'l':70, 'r':30, 't':30, 'b':70},legend=legend_dict, template='presentation')


In [ ]:
li_point_redeem = []
li_point_issue = []
connection = connect_prd()
for dr in pd.date_range(now_str, to_str, freq='30min'):
    q_3 = '''
    -- check trx with ponta redeemed and issued
    select max(sum_ponta_redeem), avg(sum_ponta_redeem), min(sum_ponta_redeem)
    from(
        select 
            tto.tbto_ponta_id, sum(ttp.tbtp_ponta_amount) as sum_ponta_redeem, count(tto.tbto_id) as count_order_id
        from 
            tb_transaction_order tto 
            left join tb_transaction_payment ttp on tto.tbto_id = ttp.tbto_id
        where 
            tto.tbto_create_date between '{shift_str}' and '{now_str}'
            and ttp.tbmp_id = 23
            and tto.tbto_status not in ('18','10','11')
            and tbto_status is not null
        group by tto.tbto_ponta_id 
        order by sum_ponta_redeem desc
    ) as s_p_r
    '''.format(shift_str=str(dr-timedelta(minutes=30)), now_str=str(dr))
    point_redeem = pd.read_sql(q_3, connection)
    point_redeem['date'] = dr
    li_point_redeem.append(point_redeem)
    

    
    q_4 = '''
    -- check trx with ponta redeem and issued
    select max(sum_ponta_issued), avg(sum_ponta_issued), min(sum_ponta_issued)
    from(
        select 
            tto.tbto_ponta_id, sum(tto.tbto_ponta_point) as sum_ponta_issued, count(tto.tbto_id) as count_order_id
        from 
            tb_transaction_order tto 
        where 
            tto.tbto_create_date between '{shift_str}' and '{now_str}'
            and tbto_ponta_id is not null
            and tto.tbto_status not in ('18','10','11')
            and tbto_status is not null
        group by tto.tbto_ponta_id 
        order by sum_ponta_issued desc
    ) as s_p_i
    '''.format(shift_str=str(dr-timedelta(minutes=30)), now_str=str(dr))
    point_issue = pd.read_sql(q_4, connection)
    point_issue['date'] = dr
    li_point_issue.append(point_issue)
    

    

connection.close()
point_redeem = pd.concat(li_point_redeem)
point_issue = pd.concat(li_point_issue)
point_redeem = point_redeem.fillna(0)
point_issue = point_issue.fillna(0)



## Use point in range 30 minutes
> 
* every ticks represent range 30 minutes
* full plot show 1 month data

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

# fig = px.line(voucher_count_check, x='date', y=voucher_count_check.iloc[:,0:2], template='seaborn', \
#               text='amount_voucher')

fig = go.Figure()
fig.add_trace(go.Scatter(
    mode = "text+lines",
    x=point_redeem['date'],
    y=point_redeem['max'],
    name='max',
#     text = point_redeem['max'].astype(int).astype(str),
    textposition='top center',


))
fig.add_trace(go.Scatter(
    mode = "text+lines",
    x=point_redeem['date'],
    y=point_redeem['avg'],
    name='avg',
#     text = point_redeem['avg'].astype(int).astype(str),
    textposition='top center',

))

mean_redeem_max = int(point_redeem['max'].mean())
fig.add_hline(y=mean_redeem_max, line_dash="dot",
              annotation_text="avg<br>{}".format(mean_redeem_max), 
              annotation_position="top right")

# fig.update_traces(texttemplate='%{text:.2s}', 
#     textposition='top center', 
#     textfont_size=11,
#     hovertemplate='%{x}<br>%{text}')

fig.update_xaxes(
    tickformat="%d%b",
    showgrid=True, gridwidth=1, gridcolor='LightPink', title=''
)
fig.update_yaxes(

    showgrid=True, gridwidth=1, gridcolor='LightPink', title='# use point'
)

legend_dict = \
    legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            title='',
            title_font_family="Times New Roman",
            font=dict(
                family="Courier",
                size=12,
                color="black"
            ),
            bgcolor="LightGrey",
            bordercolor="Black",
            borderwidth=1
        )

In [ ]:
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', margin=\
                  {'l':70, 'r':30, 't':30, 'b':70},legend=legend_dict, template='presentation')


## Get point in range 30 minutes
> 
* every ticks represent range 30 minutes
* full plot show 1 month data

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

# fig = px.line(voucher_count_check, x='date', y=voucher_count_check.iloc[:,0:2], template='seaborn', \
#               text='amount_voucher')

fig = go.Figure()
fig.add_trace(go.Scatter(
    mode = "text+lines",
    x=point_issue['date'],
    y=point_issue['max'],
    name='max',
#     text = point_issue['max'].astype(int).astype(str),
    textposition='top center',


))
fig.add_trace(go.Scatter(
    mode = "text+lines",
    x=point_issue['date'],
    y=point_issue['avg'],
    name='avg',
#     text = point_issue['avg'].astype(int).astype(str),
    textposition='top center',

))

mean_issue_max = int(point_issue['max'].mean())
fig.add_hline(y=mean_issue_max, line_dash="dot",
              annotation_text="avg<br>{}".format(mean_issue_max), 
              annotation_position="top right")


# fig.update_traces(texttemplate='%{text:.2s}', 
#     textposition='top center', 
#     textfont_size=11,
#     hovertemplate='%{x}<br>%{text}')

fig.update_xaxes(
    tickformat="%d%b",
    showgrid=True, gridwidth=1, gridcolor='LightPink', title=''
)
fig.update_yaxes(

    showgrid=True, gridwidth=1, gridcolor='LightPink', title='# get point'
)

legend_dict = \
    legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            title='',
            title_font_family="Times New Roman",
            font=dict(
                family="Courier",
                size=12,
                color="black"
            ),
            bgcolor="LightGrey",
            bordercolor="Black",
            borderwidth=1
        )

In [ ]:
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', margin=\
                  {'l':70, 'r':30, 't':30, 'b':70},legend=legend_dict, template='presentation')